In [3]:
# required libraries
import numpy as np
import math

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col, udf, to_timestamp, lit
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import when, rand
from pyspark.ml.feature import Normalizer, StandardScaler, MinMaxScaler, VectorAssembler

from tensorflow.keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate, Dropout, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#dataset path
dataset_folder_s3 = 's3://bot-dataset/data/'# 'data/' # 's3://bot-dataset/data/'
result_path_s3 = 's3://bot-dataset/result/' # '' # 's3://bot-dataset/result/'

## convert into feature vector for ml model
feature_columns = ['age', 'has_location', 'is_verified', 'total_tweets', 'total_following', 
                   'total_followers', 'total_likes', 'has_avatar', 'has_background', 
                   'is_protected', 'profile_modified']

# read dataset from csv
def read_dataset(spark):
    requiredColumns = ['screen_name', 'created_at', 'updated', 'location', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'default_profile_image', 'profile_use_background_image', 'protected', 'default_profile']

    bot_accounts1 = spark.read.csv(dataset_folder_s3 + 'social_spambots_1.csv', header = True, inferSchema = True).select(requiredColumns)
    bot_accounts2 = spark.read.csv(dataset_folder_s3 + 'social_spambots_2.csv', header = True, inferSchema = True).select(requiredColumns)
    bot_accounts3 = spark.read.csv(dataset_folder_s3 + 'social_spambots_3.csv', header = True, inferSchema = True).select(requiredColumns)

    # combine multiple bot_account dataset
    bot_accounts = bot_accounts1.union(bot_accounts2.union(bot_accounts3))
    clean_accounts = spark.read.csv(dataset_folder_s3 + 'geniune_accounts.csv', header = True, inferSchema = True).select(requiredColumns)
    
    return bot_accounts, clean_accounts

# clean dataset
def cleanData(df):
    df = df.withColumn('age', lit(0)) # need to calculate from 'updated' -'created_at'
    df = df.withColumn('has_location', when((df['location'] != None), 1).otherwise(0))
    df = df.withColumn('has_avatar', when((df['default_profile_image'] != None), 1).otherwise(0))
    df = df.withColumn('has_background', when((df['profile_use_background_image'] != None), 1).otherwise(0))
    df = df.withColumn('is_verified', when((df['verified'] != None), 1).otherwise(0))
    df = df.withColumn('is_protected', when((df['protected'] != None), 1).otherwise(0))
    df = df.withColumn('profile_modified', when((df['default_profile'] != None), 1).otherwise(0))
    df = df.withColumnRenamed("screen_name", "username")
    df = df.withColumnRenamed("statuses_count", "total_tweets")
    df = df.withColumnRenamed("friends_count", "total_following")
    df = df.withColumnRenamed("followers_count", "total_followers")
    df = df.withColumnRenamed("favourites_count", "total_likes")
    
    return df.select('username', 'age', 'has_location', 'is_verified', 'total_tweets', 'total_following', 'total_followers', 'total_likes', 'has_avatar', 'has_background', 'is_protected', 'profile_modified')



def doDataScaling(df, input_column, output_column):
    ## Make data standard
    # https://spark.apache.org/docs/1.4.1/ml-features.html#standardscaler

    scaler = StandardScaler(inputCol=input_column, outputCol=output_column,
                            withStd=True, withMean=False)

    # Compute summary statistics by fitting the StandardScaler
    scalerModel = scaler.fit(df)

    # Normalize each feature to have unit standard deviation.
    scaled_df = scalerModel.transform(df)
    
    return scaled_df


def datasetSplit(X):
    # split data for training ana testing
    train_df, test_df = X.randomSplit([0.80, 0.20])

    # features --> 'BotOrNot'
    X_train = train_df.drop('BotOrNot')
    y_train = train_df.select('BotOrNot')
    X_test = test_df.drop('BotOrNot')
    y_test = test_df.select('BotOrNot')
    
    return X_train, y_train, X_test, y_test



## create model
def getDLModel():
    model = Sequential()
    model.add(Dense(500, input_dim=11))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(200))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model


# convert DataFrame column into nparray
# nparray required for model training, validation

def to_nparray_list(df, column_name):
    rows = df.select(column_name).collect()
    lists = [x[column_name] for x in rows]
    nparr = np.array(lists)
    
    return nparr

# ml model train and validation

def saveModel(model):
    model.save(result_path_s3 + 'my_model.h5')

def app():
    # init spark
    spark = SparkSession.builder.appName('ml_account_base_session').getOrCreate()
    
    bot_accounts, clean_accounts = read_dataset(spark)
    
    bot_accounts = cleanData(bot_accounts)
    clean_accounts = cleanData(clean_accounts)
    
    ## add BotOrNot column
    bot_accounts = bot_accounts.withColumn('BotOrNot', lit(1))
    clean_accounts = clean_accounts.withColumn('BotOrNot', lit(0))
    
    #combine clean and bot accounts data togather
    combined_df = bot_accounts.union(clean_accounts)

    # shuffle dataset
    new_df = combined_df.orderBy(rand())

    #remove 'userrname' columns from dataset
    new_df = new_df.drop('username')
    
    feature_assembler = VectorAssembler(inputCols = feature_columns, outputCol = 'independent_features')
    df_updated = feature_assembler.transform(new_df)

    # keep only required features/columns
    df_updated = df_updated.select('independent_features', 'BotOrNot')
    
    scaled_df = doDataScaling(df_updated, "independent_features", "scaled_features")
    
    # keep only necessary feature/column for ml model
    XY = scaled_df.select('scaled_features', 'BotOrNot')
    
    X_train, y_train, X_test, y_test = datasetSplit(XY)
    
    # DataFrame(column) --> nparray
    X_train = to_nparray_list(X_train, 'scaled_features')
    y_train = to_nparray_list(y_train, 'BotOrNot')
    X_test = to_nparray_list(X_test, 'scaled_features')
    y_test = to_nparray_list(y_test, 'BotOrNot')
    
    model = getDLModel()
    
    model.fit(X_train, y_train,
          batch_size=64,
          epochs=20,
          validation_data=(X_test, y_test))
    
    
    score, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Test accuracy:', acc)
    
    saveModel(model)

    
if __name__ == '__main__':
    app()

    
    
    
    
    

Epoch 1/20
105/105 [==============================] - 2s 7ms/step - loss: 0.4168 - accuracy: 0.8631 - val_loss: 0.3017 - val_accuracy: 0.8938
Epoch 2/20
105/105 [==============================] - 1s 5ms/step - loss: 0.2529 - accuracy: 0.9175 - val_loss: 0.2583 - val_accuracy: 0.9142
Epoch 3/20
105/105 [==============================] - 1s 5ms/step - loss: 0.2025 - accuracy: 0.9341 - val_loss: 0.1846 - val_accuracy: 0.9418
Epoch 4/20
105/105 [==============================] - 1s 5ms/step - loss: 0.1760 - accuracy: 0.9427 - val_loss: 0.1660 - val_accuracy: 0.9496
Epoch 5/20
105/105 [==============================] - 1s 5ms/step - loss: 0.1571 - accuracy: 0.9498 - val_loss: 0.1611 - val_accuracy: 0.9472
Epoch 6/20
105/105 [==============================] - 1s 5ms/step - loss: 0.1432 - accuracy: 0.9521 - val_loss: 0.1488 - val_accuracy: 0.9556
Epoch 7/20
105/105 [==============================] - 1s 5ms/step - loss: 0.1368 - accuracy: 0.9555 - val_loss: 0.1359 - val_accuracy: 0.9556
Epoch 

NameError: name 'saveModel' is not defined